In [2]:
#brainfuck parser v2

braincleanse = """
pre = 49;
c = pre + pre;
print(c);
d = c + 16;
print(d);
c -= 1;
print(c);
d -=9;
print(d);
d += 5;
print(d);
pre -= 17;
print(pre);
c += 5;
print(c);
e = d + 7;
print(e);
c -= 3;
print(c);
d -= 3;
print(d);
pre +=1;
print(pre);
"""

dataPointer = 0
#all other are guaranteed to be 0
#@ is for temp variables
variables = {}
#ready for recycling
#currently not used
unusedCells = {}
brainfuckCommands = []

#code checks before compilation to brainfuck
def staticCheck(i):
    pass

braincleanse = braincleanse.replace('\n',"")
dirtyLines = braincleanse.split(";")
lines = []
for i in dirtyLines:
    if (i != "" and ("=" in i or "(" in i or "{" in i or "}" in i) and not "#" in i):
        i = i.replace(" ", "")
        staticCheck(i)
        lines.append(i)
        
#around dataPointer
def allocateVariable(var):
    #chance to fill previous
    cell = dataPointer - 1
    while True:
        if cell < 0:
            cell = 0
            continue
        free = True
        for i in variables:
            if variables[i] == cell:
                free = False
                break
        if free:
            realloc = False
            if cell in unusedCells:
                del(unusedCells[cell])
                
            if var in variables:
                if variables[var] != cell:
                    unusedCells[variables[var]] = None
                    realloc = True
            variables[var] = cell
            print("reallocating" if realloc else "allocating new" + " variable:", var, " at location:", cell)
            break
        cell += 1
        
#for temps
def freeVariable(var):
    del variables[var]
    print("freed variable:", var)

def parseLine(string):
    found = string.find("=")
    specialOperators = ['+', '-']
    if found != -1:
        if string[found - 1] in specialOperators:
            return (string[0:found - 1], string[found - 1:])
        else:
            return (string[0:found], string[found+1:])
    else:
        return("", string)

def isnumber(x):
    try:
        int(x)
        return True
    except ValueError:
        return False
    
def fetchData(index):
    global dataPointer
    assert(index >= 0)
    move = index - dataPointer
    dataPointer = index
    if move < 0:
        brainfuckCommands.append("<" * (move * -1))
    else:
        brainfuckCommands.append(">" * move)
        
def changeData(n):
    if n < 0:
        brainfuckCommands.append("-" * (n * -1))
    else:
        brainfuckCommands.append("+" * n)
    
###BEGIN OPERATORS###
def increment(context):
    if context[1].startswith("+="):
        context = (context[0],context[1][2:], context[2])
        #if not numeric, do this, else do '+' increment
        if isnumber(context[1]):
            assignConstant(context)
        else:
            xIndex = variables[context[1]]
            allocateVariable(context[1])
            
            fetchData(xIndex)
            brainfuckCommands.append("[")
            fetchData(variables[context[0]])
            brainfuckCommands.append("+")
            fetchData(variables[context[1]])
            brainfuckCommands.append("+")
            fetchData(xIndex)
            brainfuckCommands.append("-]")
        return True
    return False
            
def decrement(context):
    if context[1].startswith("-="):
        context = (context[0],context[1][2:], context[2])
        #if not numeric, do this, else do '-' decrement
        if isnumber(context[1]):
            if not context[1].startswith('-'):
                context = (context[0],"-" + context[1])
            else:
                context = (context[0],context[1][1:])
            assignConstant(context)
        else:
            xIndex = variables[context[1]]
            allocateVariable(context[1])
            
            fetchData(xIndex)
            brainfuckCommands.append("[")
            fetchData(variables[context[0]])
            brainfuckCommands.append("-")
            fetchData(variables[context[1]])
            brainfuckCommands.append("+")
            fetchData(xIndex)
            brainfuckCommands.append("-]")
        return True
    return False

def add(context):
    plusIndex = context[1].find("+")
    if plusIndex != -1:
        x0 = context[1][:plusIndex]
        x1 = context[1][plusIndex + 1:]
        if x0 == "" or x1 == "":
            return False
        #clear if previously held value
        if context[2]:
            fetchData(variables[context[0]])
            brainfuckCommands.append("[-]")
        increment((context[0], "+=" + x0, context[2]))
        increment((context[0], "+=" + x1, context[2]))
        return True
    return False
        
def substract(context):
    minusIndex = context[1].find("-")
    if minusIndex != -1:
        x0 = context[1][:minusIndex]
        x1 = context[1][minusIndex + 1:]
        if x0 == "" or x1 == "":
            return False
        #clear if previously held value
        if context[2]:
            fetchData(variables[context[0]])
            brainfuckCommands.append("[-]")
        increment((context[0], "+=" + x0, context[2]))
        decrement((context[0], "-=" + x1, context[2]))
        return True
    return False

def getInput(context):
    if context[1].find("input(") != -1:
        expression = context[1][6:-1]
        allocateVariable(expression)
        fetchData(variables[expression])
        brainfuckCommands.append(',')
        return True
    return False

def printOutput(context):
    if context[1].find("print(") != -1:
        expression = context[1][6:-1]
        if isnumber(expression):
            allocateVariable("@")
            fetchData(variables["@"])
            changeData(int(expression))
            freeVariable("@")
        else:
            fetchData(variables[expression])
        brainfuckCommands.append('.')
        return True
    return False

#always last!
def assignConstant(context):
    if isnumber(context[1]):
        fetchData(variables[context[0]])
        changeData(int(context[1]))
    else:
        increment((context[0], "+=" + context[1], context[2]))
    return True
###END OPERATORS###
    
for i in lines:
    parsed = parseLine(i)
    if parsed[0] != "" and parsed[0] not in variables:
        allocateVariable(parsed[0])
        parsed = (parsed[0], parsed[1], False)
    else:
        parsed = (parsed[0], parsed[1], True)
                  
    #check for possible operations
    if increment(parsed):
        continue
    if decrement(parsed):
        continue
    if add(parsed):
        continue
    if substract(parsed):
        continue
    if getInput(parsed):
        continue
    if printOutput(parsed):
        continue  
    if assignConstant(parsed):
        continue
    
brainfuck = "".join(brainfuckCommands)
print("\n\n######################OUTPUT#########################")
print(brainfuck)

allocating new variable: pre  at location: 0
allocating new variable: c  at location: 1
reallocating pre  at location: 2
reallocating pre  at location: 0
allocating new variable: d  at location: 2
reallocating c  at location: 3
allocating new variable: e  at location: 4
reallocating d  at location: 5


######################OUTPUT#########################
+++++++++++++++++++++++++++++++++++++++++++++++++[>+>+<<-]>>[<+<+>>-]<.[>+>+<<-]>++++++++++++++++.>-.<---------.+++++.<<-----------------.>>>+++++.<[>>+>+<<<-]>>+++++++.<---.>>---.<<<<<+.
